In [1]:
from logistic_agent import LogisticAgent

In [2]:
log_agent =  LogisticAgent(name="LogisticAgent")

[autogen.oai.client: 02-11 17:24:59] {261} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [3]:
message = {
            "request_type": "get_supplies",
            "parameters": {
                "location": "Mountain Base Camp",
                "team": "5 members",
                "duration": 3,
                "resources": ["medical", "shelter", "food"]
            }
        }
response = log_agent.process_request(message)

processing request


In [6]:
response['recommendations']

"Okay, here's a structured supply recommendation list for a 5-member team at a mountain base camp for 3 days, considering medical, shelter, and food requirements.\n\n**1. Essential Supplies List with Quantities:**\n\n| Item Category         | Item Description                                     | Quantity (for 5 members) | Priority | Notes                                                                              |\n|-----------------------|------------------------------------------------------|-------------------------|----------|------------------------------------------------------------------------------------|\n| **Shelter & Warmth**  | Tent (4-season, appropriate for group size)          | 1                       | High     | Ensure it can withstand potential adverse weather conditions (wind, snow, rain). |\n|                       | Sleeping Bag (rated for expected low temperatures) | 5                       | High     | Temperature rating should be at least 10 degrees lower t